# Regresión lineal multiple

El modelo matricial general es el siguiente para el modelo general de regresión lineal, en el caso de la regresión lineal simple solo existiría una columna para la variable predictora $x$ y solo dos valores de los parámetros $\theta$ ($\theta_{0}$ y $\theta_{1}$.

$Y=\large{
\left[
\begin{array}{40}
y^{(1)}\\
y^{(2)}\\
y^{(3)}\\
y^{(3)}\\
:\\
y^{(m)}
\end{array}
\right]
}$, $\large{
X=
\left[
\begin{array}{44}
1&x_{1}^1&x_{2}^1&x_{3}^1&...&x_{n}^1\\
1&x_{1}^2&x_{2}^2&x_{3}^2&...&x_{n}^2\\
1&x_{1}^3&x_{2}^3&x_{3}^3&...&x_{n}^3\\
1&x_{1}^4&x_{2}^4&x_{3}^4&...&x_{n}^3\\
:&:&:&:&:&:\\
1&x_{1}^m&x_{2}^m&x_{3}^m&...&x_{n}^m\\
\end{array}
\right]
}$ ,
$\large{
\theta=
\left[
\begin{array}{40}
\theta_{0}\\
\theta_{1}\\
\theta_{2}\\
\theta_{3}\\
:\\
\theta_{n}
\end{array}
\right]
}$
### División de los datos en set de entrenamiento y validación

In [ ]:
import numpy as np # https://numpy.org/
import pandas as pd # https://pandas.pydata.org/
import seaborn as sns # https://seaborn.pydata.org/
import matplotlib.pyplot as plt # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

In [ ]:
# Leer el archivo generado en el notebook de preproceso
url = 'https://raw.githubusercontent.com/JASDataCTG/Diplomado-ML/main/Modulo%202/Datasets/precio_prep_noparq.csv'
df = pd.read_csv(url, header = 0)

In [ ]:
# Crear un nuevo dataframe sin la columna precio (axis=1 -> borra columna)
X_multiple = df.drop('precio', axis = 1)

In [ ]:
y_multiple = df['precio']

In [ ]:
# Importar el modulo y la función necesaria para el modelo de regresión lineal
from sklearn.linear_model import LinearRegression

# Importar modulo y función necesaria para hacer una división aleatoria de los datos
from sklearn.model_selection import train_test_split 

In [ ]:
# Obtener los cuatro conjuntos de datos necesarios para el modelo
X_train, X_test, y_train, y_test = train_test_split(X_multiple, y_multiple, test_size = 0.2, random_state = 123)

In [ ]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape # Revisar las dimensiones de las matrices de los dataframes generados

Las 506 instancias del conjunto de datos se dividen en un set de entrenamiento de 404 instancias y otro de validación con 102 instancias, equivalente al 80 y 20% respectivamente. Se asigna una semilla para generar los datos con el fin de que este sea reproducible y no cambien los datos en cada ejecución de la celda del notebook.

In [ ]:
linear_model_s = LinearRegression()

In [ ]:
linear_model_s.fit(X_train, y_train)

In [ ]:
# Realizar la predicción en el conjunto de entrenamiento
y_train_s_pred = linear_model_s.predict(X_train)

In [ ]:
# Realizar la predicción en el conjunto de validación
y_test_s_pred = linear_model_s.predict(X_test)

In [ ]:
# Ver las predicciones desarrolladas por el modelo para el conjunto de validación X_test
y_test_s_pred

In [ ]:
# Ver el conjunto de validación
X_test

In [ ]:
# Convertir el array vector de predicciones a dataframe
y_test_s_df = pd.DataFrame(y_test_s_pred, columns = ['Prediccion'])

In [ ]:
y_test_s_df

In [ ]:
# Reindexar X_texst para poder hacer el join con y_test_s_df
X_test_reindex = X_test.reset_index(drop = True, inplace = False)

In [ ]:
# Realizar la fusión de los dos datafram para poder ver el las variables predictoras
# y la variable predica en un solo dataframe
test_x_con_pred = X_test_reindex.join(y_test_s_df)

In [ ]:
test_x_con_pred

In [ ]:
# Importar la función que realiza el calculo del coeficiente de determinación
# con el fin de analizar la bondad de ajuste del modelo
from sklearn.metrics import r2_score

In [ ]:
r2_score?

In [ ]:
# Ver la bondad de ajuste en los datos de entrenamiento
r2_score(y_train, y_train_s_pred)

El valor de $r^2$ del modelo nos informa que el modelo es capaz de explicar, aproximadamente, el 25% de la varianza de la variable predicha de los precios de las propiedades en el conjunto de entrenamiento.

In [ ]:
r2_score(y_test, y_test_s_pred) # Ver la bondad de ajuste en los datos de validación

El valor de $r^2$ del modelo nos informa que el modelo es capaz de explicar, aproximadamente, el 60.5% de la varianza de la variable predicha de los precios de las propiedades en el conjunto de validación.

### Validación cruzada (KFold)
![Validación cruzada](5-Fold-Cross-Validation.jpg)
El método de validación que vimos en la sección anterior se conoce como el método de validación simple, muy utilizado para conjuntos de tamaño considerable (más de 2 millones de instancias), sin embargo, cuando tenemos datasets con pocas instancias (más de 200 y menos de 2 millones) se puede utilizar una técnica conocida como *Cross Validation* o validación cruzada. La validación cruzada consiste en dividir los datos de forma aleatoria en $k$ grupos de aproximadamente el mismo tamaño, $k-1$ grupos se emplean para desarrollar el entremiento del modelo y uno de los grupos se emplea para la validación.

In [ ]:
# Importar el modulo necesario para desarrollar la validación cruzada
from sklearn.model_selection import KFold

# Importar las métricas de evaluación del modelo (score)
from sklearn.model_selection import cross_val_score

In [ ]:
# Escoger las porciones en que vamos a dividir el dataset de entrenamiento para hacer las particiones y las pruebas
kf = KFold(n_splits = 10)

In [ ]:
# Hallar el valor de r2 con la función score
score = linear_model_s.score(X_train, y_train)

In [ ]:
score

In [ ]:
# Desarrollar la validación cruzada con el conjunto de datos de entrenamiento
score_cv_train = cross_val_score(linear_model_s, X_train, y_train, cv = kf)

In [ ]:
score_cv_train

In [ ]:
# Calcular la media de los score (r cuadrado) de las diez validaciones desarrolladas
print("Media de las validaciones set de entrenamiento: ", score_cv_train.mean())

Observar que con la validación cruzada se obtiene un valor de $r^2$ menor al obtenido por la validación simple o particionada.